In [1]:
# pip install huggingface_hub transformers datasets evaluate jiwer ipywidgets accelerate

In [19]:
import torch
print(torch.cuda.is_available())

True


In [20]:
from transformers import pipeline
import torch

device = "cuda:0" if torch.cuda.is_available() else "cpu"

classifier = pipeline(
    "audio-classification", model="MIT/ast-finetuned-speech-commands-v2", device=device
)

In [21]:
#classifier.model.config.id2label

In [22]:
classifier.model.config.id2label[27]

'marvin'

In [23]:
from transformers.pipelines.audio_utils import ffmpeg_microphone_live


def launch_fn(
    wake_word="marvin",
    prob_threshold=0.5,
    chunk_length_s=2.0,
    stream_chunk_s=0.25,
    debug=False,
):
    if wake_word not in classifier.model.config.label2id.keys():
        raise ValueError(
            f"Wake word {wake_word} not in set of valid class labels, pick a wake word in the set {classifier.model.config.label2id.keys()}."
        )

    sampling_rate = classifier.feature_extractor.sampling_rate

    mic = ffmpeg_microphone_live(
        sampling_rate=sampling_rate,
        chunk_length_s=chunk_length_s,
        stream_chunk_s=stream_chunk_s,
    )

    print("Listening for wake word...")
    for prediction in classifier(mic):
        prediction = prediction[0]
        if debug:
            print(prediction)
        if prediction["label"] == wake_word:
            if prediction["score"] > prob_threshold:
                return True

In [24]:
launch_fn(debug=True)

Listening for wake word...
{'score': 0.04920288547873497, 'label': 'two'}
{'score': 0.05501312389969826, 'label': 'cat'}
{'score': 0.06954606622457504, 'label': 'eight'}
{'score': 0.2631351351737976, 'label': 'three'}
{'score': 0.33571821451187134, 'label': 'up'}
{'score': 0.3973909020423889, 'label': 'up'}
{'score': 0.3973909914493561, 'label': 'up'}
{'score': 0.397391140460968, 'label': 'up'}
{'score': 0.10213109105825424, 'label': 'up'}
{'score': 0.13102653622627258, 'label': 'up'}
{'score': 0.14298561215400696, 'label': 'up'}
{'score': 0.14298561215400696, 'label': 'up'}
{'score': 0.14298561215400696, 'label': 'up'}
{'score': 0.14298556745052338, 'label': 'up'}
{'score': 0.10517124086618423, 'label': 'off'}
{'score': 0.09166945517063141, 'label': 'off'}
{'score': 0.09166944772005081, 'label': 'off'}
{'score': 0.09166945517063141, 'label': 'off'}
{'score': 0.091669462621212, 'label': 'off'}
{'score': 0.09484037011861801, 'label': 'off'}
{'score': 0.11452919989824295, 'label': 'off'}

KeyboardInterrupt: 

 Speech transcription

In [8]:
transcriber = pipeline(
    "automatic-speech-recognition", model="openai/whisper-base.en", device=device
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
import sys


def transcribe(chunk_length_s=5.0, stream_chunk_s=1.0):
    sampling_rate = transcriber.feature_extractor.sampling_rate

    mic = ffmpeg_microphone_live(
        sampling_rate=sampling_rate,
        chunk_length_s=chunk_length_s,
        stream_chunk_s=stream_chunk_s,
    )

    print("Start speaking...")
    for item in transcriber(mic, generate_kwargs={"max_new_tokens": 128}):
        sys.stdout.write("\033[K")
        print(item["text"], end="\r")
        if not item["partial"][0]:
            break

    return item["text"]

In [10]:
transcribe()

Start speaking...


' you'

Language model query

In [11]:
from huggingface_hub import notebook_login

notebook_login()

In [12]:
from huggingface_hub import HfFolder
import requests


def query(text, model_id="tiiuae/falcon-7b-instruct"):
    api_url = f"https://api-inference.huggingface.co/models/{model_id}"
    headers = {"Authorization": f"Bearer {HfFolder().get_token()}"}
    payload = {"inputs": text}

    print(f"Querying...: {text}")
    response = requests.post(api_url, headers=headers, json=payload)
    return response.json()[0]["generated_text"][len(text) + 1 :]

In [13]:
query("What does Hugging Face do?")

Querying...: What does Hugging Face do?


'Hugging Face is a company that provides natural language processing and machine learning tools for developers. They'

Synthesise speech

In [14]:
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan

processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")

model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts").to(device)
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan").to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [15]:
from datasets import load_dataset

embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
speaker_embeddings = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)

In [16]:
def synthesise(text):
    inputs = processor(text=text, return_tensors="pt")
    speech = model.generate_speech(
        inputs["input_ids"].to(device), speaker_embeddings.to(device), vocoder=vocoder
    )
    return speech.cpu()

In [17]:
from IPython.display import Audio

audio = synthesise(
    "Hugging Face is a company that provides natural language processing and machine learning tools for developers."
)

Audio(audio, rate=16000)

In [18]:
launch_fn()
transcription = transcribe()
response = query(transcription)
audio = synthesise(response)

Audio(audio, rate=16000, autoplay=True)

Listening for wake word...
Start speaking...
Querying...:  How are you?
